# HDS 5230-07
## Week 09 Assignment
## Vivek Golla

# Part I: Data Loading and Cleaning

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets

In [2]:
## Set default figure size to be larger
## this may only work in matplotlib 2.0+!
matplotlib.rcParams['figure.figsize'] = [10.0,6.0]
## Enable multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
#Loading in the patient data
## Set print limits
pd.options.display.max_rows = 10
## Import Data
df_patient = \
 pd.read_csv('./PatientAnalyticFile.csv')
df_patient

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013-04-27,2018-06-01,NaN
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2011-11-05,2015-11-13,NaN
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-09-22,2018-06-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1997-12-19,female,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2008-06-14,2018-06-01,NaN
19996,19997,1984-03-31,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2007-04-24,2018-06-01,NaN
19997,19998,1993-07-04,female,white,0,0,0,0,0,0,...,0,0,1,0,1,0,0,2010-10-16,2018-06-01,NaN
19998,19999,1984-04-17,male,other,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2015-01-04,2018-06-01,NaN


In [5]:
# Create mortality variable
df_patient['mortality'] = \
    np.where(df_patient['DateOfDeath'].isnull(),
             0,1)

In [6]:
# Convert dateofBirth to date
df_patient['DateOfBirth'] = \
    pd.to_datetime(df_patient['DateOfBirth'])
# Calculate age in years as of 2015-01-01
df_patient['Age_years'] = \
    ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days/365.25)

In [7]:
## Create formula for all variables in model
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth',
               'Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)
formula

'mortality ~ Diabetes_with_complications + Stroke + Peptic_ulcer_disease + Hypertension + Diabetes_without_complications + Myocardial_infarction + Gender + Peripheral_vascular_disease + Renal + Cancer + Age_years + Dementia + Paralysis + Depression + Drugs + LiverMild + Race + LiverSevere + Congestive_heart_failure + Rheumatic + Obesity + Alcohol + Metastatic_solid_tumour + Pulmonary + HIV'

# Part II: Fitting models

In [8]:
## only use subset of data so models fit in reasonable time
df_patient_sub = \
    df_patient.sample(frac=0.1,
                     random_state=32)
## use Patsy to create model matrices
Y,X = dmatrices(formula,
                df_patient_sub)

In [9]:
## Split Data into training and sample
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X,
                     np.ravel(Y), # prevents dimensionality error later!
                     test_size=0.25,
                     random_state=42)

In [10]:
## import linear model
from sklearn import linear_model
## Define model parameters
## can implement penalties, but check docs for appropriate solver
clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver='liblinear') # could change to lbfgs!
## fit model using data with .fit
clf.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [11]:
## Create dict to store all these results:
result_scores = {}
## Score the Model on Training and Testing Set
result_scores['Logistic'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [12]:
## Create Function to Print Results
def get_results(x1):
    print("\n{0:20}   {1:4}    {2:4}".format('Model','Train','Test'))
    print('-------------------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<6.4}   {2:<6.4}".format(i,x1[i][0],x1[i][1]))

In [13]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 


In [14]:
#Null model
## Dummy classifier
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent',
                      random_state=0)
clf.fit(X_train, y_train)
clf.score(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

0.6466666666666666

In [15]:
## Score the Model on Training and Testing Set
result_scores['Null'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 


In [16]:
## Logistic Regression with l1 penalty
## Specify penalty directly as C = 1
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=1, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)

LogisticRegression(C=1, penalty='l1', solver='liblinear')

In [17]:
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_1'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 


In [18]:
## Logistic Regression with l1 penalty
## Specify penalty directly as C = 0.1
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=0.1, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_01'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)

LogisticRegression(C=0.1, penalty='l1', solver='liblinear')


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 


In [19]:
## Logistic Regression with l1 penalty
## Specify penalty directly as C = 10
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=10, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_10'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)

LogisticRegression(C=10, penalty='l1', solver='liblinear')


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 


In [20]:
## Select the alpha through cross validation (k-folds leave one out)
# auto generate 20 values between 1e-4 and 1e4 on log scale
clf = linear_model.LogisticRegressionCV(cv=5,
                                        Cs=20, ## takes awhile to fit 20 models!
                                        penalty='l1',
                                        solver='liblinear')
clf.fit(X_train,y_train)

LogisticRegressionCV(Cs=20, cv=5, penalty='l1', solver='liblinear')

In [21]:
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_auto'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 


In [22]:
## LASSO (L1) regression, set alpha
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
## set our transformation
scaler = preprocessing.StandardScaler()

## set our model
clf = linear_model.LogisticRegressionCV(cv=5,
                                        Cs=20, ## takes awhile to fit 20 models!
                                        penalty='l1',
                                        solver='liblinear')
## put together in pipeline
pipe1 = Pipeline([("scale", scaler),
                  ("LASSO", clf)])
pipe1.fit(X_train,y_train)

Pipeline(steps=[('scale', StandardScaler()),
                ('LASSO',
                 LogisticRegressionCV(Cs=20, cv=5, penalty='l1',
                                      solver='liblinear'))])

In [23]:
## Score the Model on Training and Testing Set
result_scores['Logistic_SL1_C_auto'] = \
            (sklearn.metrics.accuracy_score(y_train,pipe1.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,pipe1.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 


In [24]:
#### This takes awhile to run!
## try multiple polynomials with a LASSO regulaizer
## use pipeline for pre-processing
from sklearn.preprocessing import PolynomialFeatures

## set our transformation
scaler = preprocessing.StandardScaler()

## polynomials
poly_feat = PolynomialFeatures(degree=2,include_bias=False)

## set our model
clf = linear_model.LogisticRegressionCV(cv=3,
                                        Cs=5,
                                        penalty='l1',
                                        solver='liblinear')
## put together in pipeline
pipe2 = Pipeline([("scale", scaler),
                  ("poly",poly_feat),
                  ("LASSO", clf)])
pipe2.fit(X_train,y_train)

Pipeline(steps=[('scale', StandardScaler()),
                ('poly', PolynomialFeatures(include_bias=False)),
                ('LASSO',
                 LogisticRegressionCV(Cs=5, cv=3, penalty='l1',
                                      solver='liblinear'))])

In [25]:
## Score the Model on Training and Testing Set
result_scores['LogisticL1_SP_C_auto'] = \
            (sklearn.metrics.accuracy_score(y_train,pipe1.predict(X_train)),
            sklearn.metrics.accuracy_score(y_test,pipe1.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 
LogisticL1_SP_C_auto   0.7307   0.714 


In [26]:
#### Fit Random Forest
## Random Forests
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=100,
                                      max_features=10,
                                      random_state=42)
clf.fit(X_train,y_train)

RandomForestClassifier(max_features=10, random_state=42)

In [27]:
## Score the Model on Training and Testing Set
result_scores['RandomForest_noCV'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 
LogisticL1_SP_C_auto   0.7307   0.714 
RandomForest_noCV      0.9993   0.692 


# Question 1: Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.

## Answer
Looking at our 'Result Scores' table, we will first prioritize those models that performed best on the Test data, because it's most important that the model has high accuracy when it comes to new and unseen data.

Our two models, Logistic and Logistic_L1_C_10, had the best performance with an accuracy score of 0.718, meaning these models are accurate 71.8% of the time!

Now, looking at the training accuracy score, we notice that Logistic_L1_C_10 has a slightly higher Training accuracy score, so we can say that overall, the best model is the Logistic Model with applied L1 penalty and C value of 10.

# Question 2: Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

In [28]:
# Let's use df_patients (whole data)
## use Patsy to create model matrices
Y,X = dmatrices(formula,
                df_patient)

In [29]:
## Split Data into training and sample, 80% training, 20%test, use random_state=42 so it is constant
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X,
                     np.ravel(Y),
                     test_size=0.2,
                     random_state=42)

In [30]:
results = {}

In [31]:
def get_results2(x1):
    print("\n{0:20}   {1:24}   {2:24}   {3}".format('Solver Used','Training subset accuracy','Testing subset accuracy','Time Taken'))
    print('------------------------------------------------------------------------------------------------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<24.4f}   {2:<24.4f}   {3:.4f} sec".format(i, x1[i][0], x1[i][1], x1[i][2]))

In [32]:
solvers = ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

In [35]:
import time
#using a loop to fit Logistic model for each solver
for solver in solvers:

  start = time.time()
  clf = linear_model.LogisticRegression(solver=solver, max_iter=1000, penalty=None, random_state=42)
  clf.fit(X_train,y_train)
  end = time.time()

  results[solver] = \
    (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
    sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)),
     end-start)

LogisticRegression(max_iter=1000, penalty=None, random_state=42)

LogisticRegression(max_iter=1000, penalty=None, random_state=42,
                   solver='newton-cg')

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_glm/_newton_solver.py:576: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration 2. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=5.51687e-17): result may not be accurate.
  warnings.warn(


LogisticRegression(max_iter=1000, penalty=None, random_state=42,
                   solver='newton-cholesky')

LogisticRegression(max_iter=1000, penalty=None, random_state=42, solver='sag')

LogisticRegression(max_iter=1000, penalty=None, random_state=42, solver='saga')

# Question 3: Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function). Summarize your results via a table with the following structure:

In [36]:
get_results2(results)


Solver Used            Training subset accuracy   Testing subset accuracy    Time Taken
------------------------------------------------------------------------------------------------------------------------
lbfgs                  0.7483                     0.7360                     0.2301 sec
newton-cg              0.7481                     0.7355                     0.2070 sec
newton-cholesky        0.7482                     0.7362                     0.6888 sec
sag                    0.7479                     0.7358                     4.7318 sec
saga                   0.7480                     0.7360                     9.7065 sec


# Question 4: Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?

## Answer:
Based on the results, and looking primarily at the Testing Subset Accuracy, we see that Newton Cholesky solver achieved the highest accuracy of 73.62%.

However, looking at the Time Taken we see that the Newton-Cholesky solver had the execution time of 0.6888 seconds. Still, as its the most accurate with respect to Testing subset, we consider it as the best Solver.